In [ ]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

from selenium.webdriver import ChromeOptions

# = = = = = = = = = = = = = = =

option = ChromeOptions()

# = = = = = = = = = = = = = = =

# option.add_extension(Proxy.get_Proxy_Selenium('coroutine'))
option.add_experimental_option('useAutomationExtension', False)
option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
# option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument(UA.get_User_Agent_Selenium())
option.page_load_strategy = 'eager'

# = = = = = = = = = = = = = = =

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

# = = = = = = = = = = = = = = =

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(3):
            try:
                browser = Chrome(service=Service('../../../chromedriver'), options=option)

                browser.maximize_window()

                browser.get(input_.loc[i, 'Url'])
                
                # = = = = = = = = = = = = = = =
                
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                time.sleep(1)
                browser.execute_script('window.scrollTo(0, -document.body.scrollHeight);')

                # = = = = = = = = = = = = = = =
                
                wait = WebDriverWait(browser, 60)
                wait.until(EC.presence_of_element_located((By.XPATH, '//h1[@product-headline]/div[@class="h1 mb-0"]')))
                title = browser.find_element(by=By.XPATH, value='//h1[@product-headline]/div[@class="h1 mb-0"]').text.strip()

                # = = = = = = = = = = = = = = =
                
                wait.until(EC.presence_of_element_located((By.XPATH, '//h1[@product-headline]/div[@class="article-number"]')))
                subtitle = browser.find_element(by=By.XPATH, value='//h1[@product-headline]/div[@class="article-number"]').text.strip()

                # = = = = = = = = = = = = = = =
                
                wait.until(EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "product-detail__body")]/descendant::div[@class="image"]/picture/img[@saamimageloader]')))
                if 'error' in browser.find_element(by=By.XPATH, value='//div[contains(@class, "product-detail__body")]/descendant::div[@class="image"]/picture/img[@saamimageloader]').get_attribute('class'):
                    src = ''
                else:
                    src = browser.find_element(by=By.XPATH, value='//div[contains(@class, "product-detail__body")]/descendant::div[@class="image"]/picture/img[@saamimageloader]').get_attribute('src').strip()

                # = = = = = = = = = = = = = = =
                
                wait.until(EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "cell-2")]/div/dl')))
                dl = browser.find_element(by=By.XPATH, value='//div[contains(@class, "cell-2")]/div/dl')
                list_dt_1 = dl.find_elements(by=By.XPATH, value='./dt')
                list_dd_1 = dl.find_elements(by=By.XPATH, value='./dd')
                
                # = = = = = = = = = = = = = = =
                
                wait.until(EC.presence_of_element_located((By.XPATH, '//div[@id="content-section--attributes"]/descendant::div[@class="ng-star-inserted"]/saam-product-attributes/dl')))
                dl = browser.find_element(by=By.XPATH, value='//div[@id="content-section--attributes"]/descendant::div[@class="ng-star-inserted"]/saam-product-attributes/dl')
                list_dt_2 = dl.find_elements(by=By.XPATH, value='./dt')
                list_dd_2 = dl.find_elements(by=By.XPATH, value='./dd')
                
                # = = = = = = = = = = = = = = =
                
                dl = browser.find_elements(by=By.XPATH, value='//div[@id="content-section--attributes"]/descendant::div[@class="ng-star-inserted"]/saam-product-attributes/dl/following-sibling::div[contains(@class, "product-attributes__toggle-attributes")]/dl')
                if len(dl) == 0:
                    list_dt_3 = []
                    list_dd_3 = []
                else:
                    list_dt_3 = dl[0].find_elements(by=By.XPATH, value='./dt')
                    list_dd_3 = dl[0].find_elements(by=By.XPATH, value='./dd')

                # = = = = = = = = = = = = = = =
                
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                time.sleep(1)
                browser.execute_script('window.scrollTo(0, -document.body.scrollHeight);')
                
                # = = = = = = = = = = = = = = =
                
                list_make = browser.find_elements(by=By.XPATH, value='//mat-tree[@class="mat-tree cdk-tree ng-star-inserted"]/mat-tree-node[@aria-level="1" and @aria-expanded="false"]')
                length_make = len(list_make)
                list_model = browser.find_elements(by=By.XPATH, value='//mat-tree[@class="mat-tree cdk-tree ng-star-inserted"]/mat-tree-node[@aria-level="2" and @aria-expanded="false"]')
                length_model = len(list_model)
                
                # = = = = = = = = = = = = = = =
                
                while length_make != 0:
                    browser.execute_script('arguments[0].click();', list_make[0].find_element(by=By.XPATH, value='./div/mat-icon'))
                    time.sleep(1)
                    list_make = browser.find_elements(by=By.XPATH, value='//mat-tree[@class="mat-tree cdk-tree ng-star-inserted"]/mat-tree-node[@aria-level="1" and @aria-expanded="false"]')
                    list_model = browser.find_elements(by=By.XPATH, value='//mat-tree[@class="mat-tree cdk-tree ng-star-inserted"]/mat-tree-node[@aria-level="2" and @aria-expanded="false"]')
                    while len(list_make) == list_make or len(list_model) == length_model:
                        time.sleep(1)
                        list_make = browser.find_elements(by=By.XPATH, value='//mat-tree[@class="mat-tree cdk-tree ng-star-inserted"]/mat-tree-node[@aria-level="1" and @aria-expanded="false"]')
                        list_model = browser.find_elements(by=By.XPATH, value='//mat-tree[@class="mat-tree cdk-tree ng-star-inserted"]/mat-tree-node[@aria-level="2" and @aria-expanded="false"]')
                    length_make = len(list_make)
                    length_model = len(list_model)
                    
                # = = = = = = = = = = = = = = =
                
                list_engine = browser.find_elements(by=By.XPATH, value='//mat-tree[@class="mat-tree cdk-tree ng-star-inserted"]/mat-tree-node[@aria-level="3" and @aria-expanded="false"]')
                length_engine = len(list_engine)
                
                # = = = = = = = = = = = = = = =
                
                while length_model != 0:
                    browser.execute_script('arguments[0].click();', list_model[0].find_element(by=By.XPATH, value='./div/mat-icon'))
                    time.sleep(1)
                    list_model = browser.find_elements(by=By.XPATH, value='//mat-tree[@class="mat-tree cdk-tree ng-star-inserted"]/mat-tree-node[@aria-level="2" and @aria-expanded="false"]')
                    list_engine = browser.find_elements(by=By.XPATH, value='//mat-tree[@class="mat-tree cdk-tree ng-star-inserted"]/mat-tree-node[@aria-level="3" and @aria-expanded="false"]')
                    while len(list_model) == length_model or len(list_engine) == length_engine:
                        time.sleep(1)
                        list_model = browser.find_elements(by=By.XPATH, value='//mat-tree[@class="mat-tree cdk-tree ng-star-inserted"]/mat-tree-node[@aria-level="2" and @aria-expanded="false"]')
                        list_engine = browser.find_elements(by=By.XPATH, value='//mat-tree[@class="mat-tree cdk-tree ng-star-inserted"]/mat-tree-node[@aria-level="3" and @aria-expanded="false"]')
                    length_model = len(list_model)
                    length_engine = len(list_engine)
                    
                # = = = = = = = = = = = = = = =
                
                list_node = browser.find_elements(by=By.XPATH, value='//mat-tree[@class="mat-tree cdk-tree ng-star-inserted"]/mat-tree-node')

                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                vehicle_status = 'ok'
                make = ''
                model = ''
                for node in list_node:
                    level = node.get_attribute('aria-level').strip()
                    if level == '1':
                        if node.get_attribute('aria-expanded').strip() != 'true':
                            vehicle_status = 'error'
                            break
                        make = node.find_element(by=By.XPATH, value='./div').text.strip()
                    elif level == '2':
                        if node.get_attribute('aria-expanded').strip() != 'true':
                            vehicle_status = 'error'
                            break
                        model = node.find_element(by=By.XPATH, value='./div').text.strip()
                    elif level == '3':
                        table = node.find_element(by=By.XPATH, value='./descendant::table')
                        list_header = table.find_elements(by=By.XPATH, value='./thead/tr/th')
                        for j in range(len(list_header)):
                            if 'Type' in list_header[j].text.strip():
                                col_type = j
                            elif 'Year' in list_header[j].text.strip():
                                col_year = j
                        list_row = table.find_elements(by=By.XPATH, value='./tbody/tr')
                        for row in list_row:
                            type_ = row.find_elements(by=By.XPATH, value='./td')[col_type].text.strip()
                            year = row.find_elements(by=By.XPATH, value='./td')[col_year].text.strip()
                            if 'from' in year:
                                list_year = [int(year.split('.')[1].strip())]
                            else:
                                list_year = year.split('-')
                                for j in range(len(list_year)):
                                    list_year[j] = int(list_year[j].split('.')[1].strip())
                            for year in list_year:
                                list_vehicle.append({'make': make, 'model': model, 'type': type_, 'year': year})

                # = = = = = = = = = = = = = = =
                
                if vehicle_status == 'error':
                    continue
                
                # = = = = = = = = = = = = = = =
                
                dict_vehicle_1 = {}
                for vehicle in list_vehicle:
                    if vehicle['make'] not in dict_vehicle_1:
                        dict_vehicle_1[vehicle['make']] = {vehicle['model']: [vehicle['year']]}
                    else:
                        if vehicle['model'] not in dict_vehicle_1[vehicle['make']]:
                            dict_vehicle_1[vehicle['make']][vehicle['model']] = [vehicle['year']]
                        else:
                            dict_vehicle_1[vehicle['make']][vehicle['model']].append(vehicle['year'])

                # = = = = = = = = = = = = = = =
                
                dict_vehicle_2 = {}
                for vehicle in list_vehicle:
                    if vehicle['make'] not in dict_vehicle_2:
                        dict_vehicle_2[vehicle['make']] = {vehicle['model']: {vehicle['type']: [vehicle['year']]}}
                    else:
                        if vehicle['model'] not in dict_vehicle_2[vehicle['make']]:
                            dict_vehicle_2[vehicle['make']][vehicle['model']] = {vehicle['type']: [vehicle['year']]}
                        else:
                            if vehicle['type'] not in dict_vehicle_2[vehicle['make']][vehicle['model']]:
                                dict_vehicle_2[vehicle['make']][vehicle['model']][vehicle['type']] = [vehicle['year']]
                            else:
                                dict_vehicle_2[vehicle['make']][vehicle['model']][vehicle['type']].append(vehicle['year'])

                # = = = = = = = = = = = = = = =
                
                list_vehicle_1 = []
                for make in dict_vehicle_1:
                    for model in dict_vehicle_1[make]:
                        year_max = max(dict_vehicle_1[make][model])
                        year_min = min(dict_vehicle_1[make][model])
                        if year_max == year_min:
                            list_vehicle_1.append(make + ' ' + model + ' ' + str(year_min))
                        else:
                            list_vehicle_1.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                vehicle_1 = '\n'.join(list_vehicle_1)
                
                # = = = = = = = = = = = = = = =
                
                list_vehicle_2 = []
                for make in dict_vehicle_2:
                    for model in dict_vehicle_2[make]:
                        for type_ in dict_vehicle_2[make][model]:
                            year_max = max(dict_vehicle_2[make][model][type_])
                            year_min = min(dict_vehicle_2[make][model][type_])
                            if year_max == year_min:
                                list_vehicle_2.append(make + ' ' + model + ' ' + str(year_min) + ' - ' + type_)
                            else:
                                list_vehicle_2.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' - ' + type_)

                vehicle_2 = '\n'.join(list_vehicle_2)
                
                # = = = = = = = = = = = = = = =
                
                list_oe = [oe.get_attribute('textContent').strip() for oe in browser.find_elements(by=By.XPATH, value='//saam-product-oe-numbers[@class="saam-product-oe-numbers ng-star-inserted"]/descendant::div[@class="mat-list-item-content"]/a')]

                oe = ';'.join(list(set(list_oe)))
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': '',
                                         'Title': title,
                                         'Subtitle': subtitle,
                                         'Vehicle_1': vehicle_1,
                                         'Vehicle_2': vehicle_2,
                                         'OE': oe,
                                         'Pic': '',
                                         'Url': input_.loc[i, 'Url'],
                                         'Src': src}])

                for dt, dd in zip(list_dt_1, list_dd_1):
                    df_temp.loc[0, dt.text.strip()] = dd.text.strip()

                for dt, dd in zip(list_dt_2, list_dd_2):
                    df_temp.loc[0, dt.text.strip().lower()] = dd.text.strip()

                for dt, dd in zip(list_dt_3, list_dd_3):
                    df_temp.loc[0, dt.get_attribute('textContent').strip().lower()] = dd.get_attribute('textContent').strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                browser.quit()
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
                
            except:
                browser.quit()
                time.sleep(0.3)
                continue

        if crawler_status != 'ok':
            df_temp = pd.DataFrame([{'Url': input_.loc[i, 'Url'],
                                     'No.': int(input_.loc[i, 'No.'])}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[i, 'Url'] +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(7):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Title'], ascending=[True])
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Url'], ascending=[True])
    output_error.to_excel('./part_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：1

